In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

spark = SparkSession.builder.appName('Puzzle').getOrCreate()

In [2]:
df = spark.read.csv('data/puzzle_040.csv', header=True, inferSchema=True)
df.show()

+-----+--------+---------+
|EmpID| EmpName|ReportsTo|
+-----+--------+---------+
|    1|   Jacob|     NULL|
|    2|     Rui|     NULL|
|    3|Jacobson|     NULL|
|    4|    Jess|        1|
|    5|   Steve|        1|
|    6|     Bob|        1|
|    7|   Smith|        2|
|    8|  Bobbey|        2|
|    9|  Steffi|        3|
|   10|  Bracha|        3|
|   11|    John|        5|
|   12| Michael|        6|
|   13|    Paul|        6|
|   14|    Lana|        7|
|   15| Johnson|        7|
|   16|     Mic|        8|
|   17|    Stev|        8|
|   18| Paulson|        9|
|   19| Jessica|       10|
+-----+--------+---------+



In [6]:
resdf= df
hirarchy = 0
resdf.alias('a').join(resdf.alias('b'), on= F.col('a.EmpID')==F.col('b.ReportsTo')).select('b.*').show()

+-----+-------+---------+
|EmpID|EmpName|ReportsTo|
+-----+-------+---------+
|    4|   Jess|        1|
|    5|  Steve|        1|
|    6|    Bob|        1|
|    7|  Smith|        2|
|    8| Bobbey|        2|
|    9| Steffi|        3|
|   10| Bracha|        3|
|   11|   John|        5|
|   12|Michael|        6|
|   13|   Paul|        6|
|   14|   Lana|        7|
|   15|Johnson|        7|
|   16|    Mic|        8|
|   17|   Stev|        8|
|   18|Paulson|        9|
|   19|Jessica|       10|
+-----+-------+---------+



In [17]:
rd = resdf.select(
    '*',
    F.lit(hirarchy).alias('HirarchyLevel'),
    F.col('EmpName').alias('FullyQualifiedName')
).where(F.col('ReportsTo')== 'NULL')

In [25]:
rd.unionAll(rd.alias('a').join(resdf.alias('b'), on= F.col('a.EmpID')==F.col('b.ReportsTo')).select('b.*', F.lit(1).alias('HirarchyLevel'),
    F.concat_ws('.', F.col('a.EmpName'), F.col('b.EmpName')).alias('FullyQualifiedName'))).show()

+-----+--------+---------+-------------+------------------+
|EmpID| EmpName|ReportsTo|HirarchyLevel|FullyQualifiedName|
+-----+--------+---------+-------------+------------------+
|    1|   Jacob|     NULL|            0|             Jacob|
|    2|     Rui|     NULL|            0|               Rui|
|    3|Jacobson|     NULL|            0|          Jacobson|
|    4|    Jess|        1|            1|        Jacob.Jess|
|    5|   Steve|        1|            1|       Jacob.Steve|
|    6|     Bob|        1|            1|         Jacob.Bob|
|    7|   Smith|        2|            1|         Rui.Smith|
|    8|  Bobbey|        2|            1|        Rui.Bobbey|
|    9|  Steffi|        3|            1|   Jacobson.Steffi|
|   10|  Bracha|        3|            1|   Jacobson.Bracha|
+-----+--------+---------+-------------+------------------+



In [13]:
rd.show()

+-----+--------+---------+-------------+------------------+
|EmpID| EmpName|ReportsTo|HirarchyLevel|FullyQualifiedName|
+-----+--------+---------+-------------+------------------+
|    1|   Jacob|     NULL|            0|             Jacob|
|    2|     Rui|     NULL|            0|               Rui|
|    3|Jacobson|     NULL|            0|          Jacobson|
|    4|    Jess|        1|            1|        Jacob.Jess|
|    5|   Steve|        1|            1|       Jacob.Steve|
|    6|     Bob|        1|            1|         Jacob.Bob|
|    7|   Smith|        2|            1|         Rui.Smith|
|    8|  Bobbey|        2|            1|        Rui.Bobbey|
|    9|  Steffi|        3|            1|   Jacobson.Steffi|
|   10|  Bracha|        3|            1|   Jacobson.Bracha|
+-----+--------+---------+-------------+------------------+



In [16]:
rd.show()

+-----+--------+---------+-------------+------------------+
|EmpID|EmpName |ReportsTo|HirarchyLevel|FullyQualifiedName|
+-----+--------+---------+-------------+------------------+
|1    |Jacob   |NULL     |0            |Jacob             |
|2    |Rui     |NULL     |0            |Rui               |
|3    |Jacobson|NULL     |0            |Jacobson          |
|4    |Jess    |1        |1            |Jacob.Jess        |
|5    |Steve   |1        |1            |Jacob.Steve       |
|6    |Bob     |1        |1            |Jacob.Bob         |
|7    |Smith   |2        |1            |Rui.Smith         |
|8    |Bobbey  |2        |1            |Rui.Bobbey        |
|9    |Steffi  |3        |1            |Jacobson.Steffi   |
|10   |Bracha  |3        |1            |Jacobson.Bracha   |
|6    |Bob     |1        |1            |Jacob.Bob         |
|5    |Steve   |1        |1            |Jacob.Steve       |
|4    |Jess    |1        |1            |Jacob.Jess        |
|8    |Bobbey  |2        |1            |